In [1]:
from py_vncorenlp import VnCoreNLP
import numpy as np 
from collections import defaultdict
from tqdm import tqdm

In [2]:
model = VnCoreNLP(
    save_dir=r"C:\Users\Admin\Documents\2024.2\NLP\Project\VnCoreNLP-master",
    max_heap_size='-Xmx2g',
    annotators=["wseg"]
)

In [3]:
import json
import re
# [
# {
#     "law_id": "01/2009/tt-bnn",
#     "articles": [
#         {
#             "article_id": "1",
#             "title": "Điều 1. Phạm vi áp dụng",
#             "text": "Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.",
#             "processed_in4": "Phạm_vi áp_dụng Thông_tư hướng_dẫn tuần_tra , canh_gác bảo_vệ đê Điều mùa lũ tuyến đê sông phân_loại , phân_cấp quy_định Điều 4 Luật Đê_Điều ."
#         },

sentences =[]
with open(r"C:\Users\Admin\Documents\2024.2\NLP\Project\dataset\legal_corpus.json", encoding = 'utf-8') as f:
    data = json.load(f)
    for doc in data:
        for article in doc['articles']:
            tokens = re.sub(r'[^\w\s_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơưăạ-ỹ]', '', article['processed_in4']).split()
            sentences.append(tokens)

In [4]:
print(sentences[1])

['Tổ_chức', 'lực_lượng', '1', 'Hàng', 'mùa', 'mưa', 'lũ', 'Uỷ_ban_nhân_dân', 'xã', 'đê', 'tổ_chức', 'lực_lượng', 'lao_động', 'địa_phương', 'tuần_tra', 'canh_gác', 'đê', 'thường_trực', 'điếm_canh', 'đê', 'dân', 'khu_vực', 'đê', 'khu_vực', 'điếm_canh', 'đê', 'báo_động', 'lũ', 'I', 'trở', 'tuyến', 'sông', 'đê', 'gọi', 'tắt', 'lực_lượng', 'tuần_tra', 'canh_gác', 'đê', '2', 'Lực_lượng', 'tuần_tra', 'canh_gác', 'đê', 'tổ_chức', 'thành', 'đội', 'Uỷ_ban_nhân_dân', 'xã', 'quyết_định', 'thành_lập', '01', '02', 'kilômét', 'đê', 'thành_lập', '01', 'đội', 'đội', '12', '18', '01', 'đội_trưởng', '01', '02', 'đội', 'phó', 'Danh_sách', 'thành_viên', 'đội', 'tuần_tra', 'canh_gác', 'đê', 'niêm_yết', 'điếm_canh', 'đê', 'địa_bàn', 'phân_công', '3', 'Khi', 'lũ', 'bão', 'diễn_biến', 'phức_tạp', 'kéo_dài', 'Uỷ_ban_nhân_dân', 'xã', 'quyết_định', 'bổ_sung', 'thành_viên', 'đội', 'tuần_tra', 'canh_gác', 'đê']


# 1. Coccurence - Matrix

In [5]:
def Vocab(sentences, min_count = 1):
    vocab = defaultdict(int)
    for sentence in sentences:
        for word in sentence:
            vocab[word] += 1
    # Only keep work with the occurence > min_count = 1
    vocab = {w : i for i, (w,c) in enumerate(vocab.items()) if c >= min_count}
    return vocab

In [6]:
def cocurrence_matrix(sentences, vocab, window_size = 5):
    matrix = defaultdict(lambda : defaultdict(float))
    for sentence in tqdm(sentences):
        sentence = [w for w in sentence if w in vocab]
        for i, word in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence) - 1, i + window_size)):
                if i != j:
                    word_i = vocab[word] #map: word -> int
                    word_j = vocab[sentence[j]]
                    distance = abs(i - j)
                    matrix[word_i][word_j] += 1.0/distance #if the word is nearer, it is more important
    return matrix

# 2. Vector initialization

In [7]:
def initialize_params(vocab_size, embedding_dim):
    # we have vocab_size word, each word is represented by a vector shape (embedding_dim)
    w = np.random.rand(vocab_size, embedding_dim)/ np.sqrt(embedding_dim)
    w_tilde = np.random.rand(vocab_size, embedding_dim)/ np.sqrt(embedding_dim)
    b = np.zeros(vocab_size)
    b_tilde = np.zeros(vocab_size)

    # Gradient accumulators (AdaGrad)
    gradsq_w = np.ones_like(w)
    gradsq_w_tilde = np.ones_like(w_tilde)
    gradsq_b = np.ones_like(b)
    gradsq_b_tilde = np.ones_like(b_tilde)

    return w, w_tilde, b, b_tilde, gradsq_w, gradsq_w_tilde, gradsq_b, gradsq_b_tilde


In [8]:
def weighting_fn(x, x_max = 100, alpha = 0.75):
    return (x/x_max)**alpha if x < x_max else 1

In [9]:
import math

# 3. Training

In [10]:
def train_glove(matrix, vocab_size, embedding_dim=50, epochs=25, x_max=100, alpha=0.75, learning_rate=0.05, patience=3):
    W, W_tilde, b, b_tilde, gradsq_W, gradsq_W_tilde, gradsq_b, gradsq_b_tilde = initialize_params(vocab_size, embedding_dim)

    best_loss = float('inf')
    best_params = None
    epochs_without_improvement = 0

    for epoch in range(epochs):
        total_loss = 0
        for i in tqdm(matrix):
            for j in matrix[i]:
                X_ij = matrix[i][j]
                weight = weighting_fn(X_ij, x_max, alpha)
                inner_product = np.dot(W[i], W_tilde[j])
                loss = inner_product + b[i] + b_tilde[j] - math.log(X_ij)
                weighted_loss = weight * loss

                total_loss += 0.5 * weighted_loss ** 2

                grad = weighted_loss
                grad_W_i = grad * W_tilde[j]
                grad_W_tilde_j = grad * W[i]

                # AdaGrad update
                W[i] -= learning_rate * grad_W_i / np.sqrt(gradsq_W[i] + 1e-8)
                W_tilde[j] -= learning_rate * grad_W_tilde_j / np.sqrt(gradsq_W_tilde[j] + 1e-8)
                b[i] -= learning_rate * grad / np.sqrt(gradsq_b[i] + 1e-8)
                b_tilde[j] -= learning_rate * grad / np.sqrt(gradsq_b_tilde[j] + 1e-8)

                # Update squared gradients
                gradsq_W[i] += grad_W_i ** 2
                gradsq_W_tilde[j] += grad_W_tilde_j ** 2
                gradsq_b[i] += grad ** 2
                gradsq_b_tilde[j] += grad ** 2

        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

        # Early stopping logic
        if total_loss < best_loss:
            best_loss = total_loss
            best_params = (W.copy(), W_tilde.copy(), b.copy(), b_tilde.copy())
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Stopping early at epoch {epoch+1} due to no improvement.")
                break

    W, W_tilde, _, _ = best_params  # Restore best weights
    return W + W_tilde

In [11]:
vocab = Vocab(sentences, min_count = 1)

In [12]:
print(len(vocab))

33961


In [13]:
matrix = cocurrence_matrix(sentences, vocab, window_size = 5)

100%|██████████| 61425/61425 [00:20<00:00, 3000.30it/s]


In [ ]:
vectors = train_glove(matrix, vocab_size=len(vocab), embedding_dim=100, epochs=100)

100%|██████████| 33961/33961 [03:28<00:00, 162.52it/s] 


Epoch 1, Loss: 163909.6510


100%|██████████| 33961/33961 [03:03<00:00, 184.95it/s] 


Epoch 2, Loss: 146290.2717


100%|██████████| 33961/33961 [01:02<00:00, 539.71it/s] 


...

100%|██████████| 33961/33961 [01:01<00:00, 555.67it/s] 


Epoch 73, Loss: 12998.2330


100%|██████████| 33961/33961 [01:00<00:00, 559.77it/s] 


Epoch 74, Loss: 12943.8296


 10%|▉         | 3251/33961 [00:38<03:10, 161.56it/s]